# Installation
You do not have to follow our installation instructions if you have roughly equivalent setups / environments already.

We will use Conda and Pip to help us install packages for this homework. If you do not have Miniconda or Anaconda, you can install Miniconda from here https://docs.conda.io/en/latest/miniconda.html.

```
conda create --name exercise2 python=3.7
conda activate exercise2

pip install jupyter
```

Go to https://pytorch.org/ to install PyTorch if you don't have it already

To install the Hugging Face `transformers` library, run
```
pip install transformers
```

Follow the instructions from https://docs.dgl.ai/en/0.4.x/install/ to install Deep Graph Library (DGL).

Spin up jupyter notebook with
```
jupyter notebook
```

# Overview
Here we illustrate components of the paper [Graph-to-Tree Learning for Solving Math Word Problems](https://www.aclweb.org/anthology/2020.acl-main.362.pdf), which solves math word problems in the MAWPS dataset. The overall pipeline looks like this (note that we replaced the BiLSTM base model in the paper with a transformer base model):

<img src="resources/workflow.png">

# `util.setup`: Input Processing
The `util.setup` function runs the preprocessing pipeline: loading in the dataset, parsing the dataset, building the vocabulary used for the models, converting the infix equation notation to prefix notation, and (if we're using T5) converting the word-level tokenization to byte-pair tokenization. We will illustrate the details here.

## Loading in Data

In [1]:
from collections import Counter
import itertools
import json
import re

import numpy as np

from util import *

path = 'data/train.json' # Path to the dataset
n_min_vocab = 5
seed = 0
val_split = 0.1

In [2]:
with open(path, 'r') as f:
    data = json.load(f)
print(len(data))

2136


## Data Format

In [3]:
data[919]

{'expression': '(7.0*(7.0+2.0))',
 'quant_cell_positions': [2, 3, 4, 10, 11, 12, 13, 14, 15, 25, 26, 27],
 'processed_question': 'A waiter had 7 tables he was waiting on , with 7 women and 2 men at each table . How many customers total did the waiter have ?',
 'raw_question': ' A waiter had 7 tables he was waiting on, with 7 women and 2 men at each table. How many customers total did the waiter have? ',
 'is_quadratic': False,
 'Id': 1025,
 'Expected': 63.0}

Each of the `2136` questions in the training set (sampled from the `2373` total questions) consists of the following five fields
1. `raw_question` is the original question.
2. `processed_question` is a processed version of the question. Each word, quantity, or punctuation is separated by a space.
3. `expression` is the desired output of the model. For the training set, this is the target. For the test set, this is compared with the prediction to score the prediction.
4. `quant_cell_positions` is a list of positions in the text corresponding to quantity cells (quantities or associated nouns, adjectives, or verbs). We provide an illustration below and you can see the paper for more detail.
5. `is_quadratic` is a flag denoting whether the problem requires solving a quadratic equation. This method cannot handle quadratics, so we discard the quadratic problems from the training set and count them as incorrect predictions for the test set. See below for an example

In [4]:
d = data[919]
tokens = np.array(d['processed_question'].split(' '))
print('Question:', d['raw_question'])
print()
print('All tokens:', tokens)
print()
print('Quantity cell positions:', d['quant_cell_positions'])
print()
print('Quantity cell tokens:', tokens[d['quant_cell_positions']])

Question:  A waiter had 7 tables he was waiting on, with 7 women and 2 men at each table. How many customers total did the waiter have? 

All tokens: ['A' 'waiter' 'had' '7' 'tables' 'he' 'was' 'waiting' 'on' ',' 'with' '7'
 'women' 'and' '2' 'men' 'at' 'each' 'table' '.' 'How' 'many' 'customers'
 'total' 'did' 'the' 'waiter' 'have' '?']

Quantity cell positions: [2, 3, 4, 10, 11, 12, 13, 14, 15, 25, 26, 27]

Quantity cell tokens: ['had' '7' 'tables' 'with' '7' 'women' 'and' '2' 'men' 'the' 'waiter'
 'have']


Here's an example of a question which requires solving the quadratic equation. We don't attempt to predict these questions and simply mark them as incorrect. Fortunately there are not very many quadratic questions in the MAWPS dataset.

In [5]:
d_quad = [d for d in data if d['is_quadratic']][0]
print('Question:', d_quad['raw_question'])
print()
print('Equation:', d_quad['expression'])

Question: A blimp flies at 60 miles per hour. A round trip of 40 miles into the wind and 40 miles with the wind takes 1.5 hours. What is the speed of the wind , in miles per hour? 

Equation: (40.0/(60.0+x))+(40.0/(60.0-x))=1.5


## Parsing Quantities

In [6]:
constants, n_max_nP = tokenize_and_separate_quants(data, n_min_vocab)

We tokenize the `processed_question` field by splitting on the spaces. We replace the quantities in the tokens by the word `'NUM'`. We put the quantities into a list called `d['nP']`.

In [7]:
d = data[919]
print('Tokens with NUM:', d['in_tokens'])
print()
print('Quantities (nP) in the input:', d['nP'])

Tokens with NUM: ['A', 'waiter', 'had', 'NUM', 'tables', 'he', 'was', 'waiting', 'on', ',', 'with', 'NUM', 'women', 'and', 'NUM', 'men', 'at', 'each', 'table', '.', 'How', 'many', 'customers', 'total', 'did', 'the', 'waiter', 'have', '?']

Quantities (nP) in the input: ['7' '7' '2']


We tokenize the `expression` field into operators and quantities. Note that if the quantity is found in `d['nP']`, we replace the quantity by a tuple containing its matching indices in `d['nP']`. In the example below, `7.0` corresponds to indices `0` and `1` in `d['nP']`, while `2.0` corresponds to index `2`.

In [8]:
print('Expression:', d['expression'])
print()
print('Out tokens:', d['out_tokens'])

Expression: (7.0*(7.0+2.0))

Out tokens: ['(', (0, 1), '*', '(', (0, 1), '+', (2,), ')', ')']


For the quantities that appears in the output of a question but not the input, we denote these as "constants".

In [9]:
constants

['0.01', '12', '1', '0.25', '100', '4', '0.5', '3', '2', '7']

Often times constants represent some implicit quantity. For example, `'0.01'` is a constant below. It appears in the output expression because it corresponds to the "%" sign in the input.

In [10]:
d = data[4]
print('Question:', d['processed_question'])
print()
print('Expression:', d['expression'])
print()
print('Out tokens:', d['out_tokens'])

Question: You deposit 70 dollars in a savings account that pays an annual interest rate of 3 % . How much simple interest would you earn in 2.5 years , in dollars ?

Expression: 70.0*2.5*3.0*0.01

Out tokens: [(0,), '*', (2,), '*', (1,), '*', '0.01']


## Splitting into Training and Test Data

In [11]:
np.random.seed(seed)
np.random.shuffle(data)
n_val = int(val_split * len(data))
train_data, val_data = data[:-n_val], data[-n_val:]

## Building Input and Output Vocab
We replace any token with fewer than `n_min_vocab` occurances in the dataset with the `<unk>` token. We build the input vocab from all the `in_tokens` in the training set along with the `<unk>` and `<pad>` tokens. The output vocab consists of the operator tokens, constants, and tokens denoting indices in `d['nP']` (we'll explain more on this shortly), along with the `<unk>` and `<pad>` tokens. 

In [12]:
default_tokens = ['<pad>', '<unk>']
operation_tokens = ['+', '-', '*', '/']

in_counts = Counter()
for d in train_data:
    in_counts.update(d['in_tokens'])
in_vocab = Vocabulary([w for w, c in in_counts.items() if c >= n_min_vocab] + default_tokens)

out_vocab = Vocabulary(operation_tokens + constants + [(i,) for i in range(n_max_nP)] + default_tokens)
out_vocab.constants = constants
out_vocab.n_constants = len(constants)
out_vocab.n_ops = len(operation_tokens)
out_vocab.base_op = 0
out_vocab.base_quant = out_vocab.base_constant = out_vocab.base_op + out_vocab.n_ops
out_vocab.base_nP = out_vocab.base_constant + out_vocab.n_constants

In [13]:
print(out_vocab.idx2token)

['+', '-', '*', '/', '0.01', '12', '1', '0.25', '100', '4', '0.5', '3', '2', '7', (0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), '<pad>', '<unk>']


Notice how `(0,)`, `(1,)`, `(2,)`, `(3,)`, `(4,)`, `(5,)`, `(6,)`, `(7,)`, and `(8,)` appear in `out_vocab`. These are actually very important for the tree decoding process for generating the output. Imagine if your input contained a quantity `1.324`; this quantity would be very rarely seen, so it doesn't make sense to add it as a constant (adding a lot of constants would make our vocab very large in size, which leads to overfitting). However, if `1.324` is the fifth quantity in the question, our model can predict `(4,)`, which represents the fifth quantity token in the input.

## Convert Infix to Prefix Notation
If you're not familiar with prefix notation, you can read more about it [here](http://www.cs.man.ac.uk/~pjj/cs212/fix.html).

In [14]:
d = data[2]
print('Infix notation:', d['out_tokens'])
print()
print('Prefix notation:', infix_to_prefix(d['out_tokens']))

Infix notation: ['(', '(', (0,), '+', (1,), ')', '-', (2,), ')']

Prefix notation: ['-', '+', (0,), (1,), (2,)]


## Converting Word-level Tokens to Byte-pair Tokens for T5
If we are using the T5 model (and not the custom `TransformerBlock` layers), we must take an additional step to use the same tokenization that the pre-trained T5 model uses. This is pretty tricky since we also have to convert the positions of the numerical tokens from the old tokenization to the new tokenization scheme, but we have written a function to make the conversion. The input vocabulary will also be the same vocab that T5 uses.

In [15]:
use_t5 = 'small'
from transformers import T5Tokenizer, T5Model
# https://arxiv.org/pdf/1910.10683.pdf
# https://huggingface.co/transformers/model_doc/t5.html
# https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_t5.py

t5_tokenizer = T5Tokenizer.from_pretrained(f't5-{use_t5}')
t5_model = T5Model.from_pretrained(f't5-{use_t5}')
in_vocab = Vocabulary(
    [k for k, v in sorted(t5_tokenizer.get_vocab().items(), key=lambda k: k[1])],
    t5_tokenizer.pad_token,
    t5_tokenizer.unk_token
)

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
d = data[105]
print('Raw question:', d['raw_question'])
print()
print('Word-level tokens:', d['in_tokens'])
print('Word-level quantity positions:', d['nP_positions'])
print('Word-level quantity cell positions:', d['quant_cell_positions'])
print()
d_t5 = d.copy()
convert_word_to_bytepair_tokenization(d_t5, t5_tokenizer)
print('T5 byte-pair tokens:', d_t5['in_tokens'])
print('T5 byte-pair quantity positions:', d_t5['nP_positions'])
print('T5 byte-pair quantity cell positions:', d_t5['quant_cell_positions'])

Raw question: On Friday, Sam spent 2 pennies on ice cream. The next day, Sam spent 12 dimes on baseball cards. All in all, how much money did Sam spend?

Word-level tokens: ['On', 'Friday', ',', 'Sam', 'spent', 'NUM', 'pennies', 'on', 'ice', 'cream', '.', 'The', 'next', 'day', ',', 'Sam', 'spent', 'NUM', 'dimes', 'on', 'baseball', 'cards', '.', 'All', 'in', 'all', ',', 'how', 'much', 'money', 'did', 'Sam', 'spend', '?']
Word-level quantity positions: [ 5 17]
Word-level quantity cell positions: [4, 5, 6, 16, 17, 18, 30, 31, 32]

T5 byte-pair tokens: ['On', 'Friday', ',', 'Sam', 'spent', '2', 'pen', 'nies', 'on', 'ice', 'cream', '.', 'The', 'next', 'day', ',', 'Sam', 'spent', '12', 'di', 'mes', 'on', 'baseball', 'cards', '.', 'All', 'in', 'all', ',', 'how', 'much', 'money', 'did', 'Sam', 'spend', '?']
T5 byte-pair quantity positions: [5, 18]
T5 byte-pair quantity cell positions: [4, 5, 6, 7, 17, 18, 19, 20, 32, 33, 34]


Notice how the quantity and quantity cell positions shifted slightly in the new tokenization scheme.

# `util.check_match`: Output Processing
Output processing is used in `util.check_match`, which evaluates predicted tokens against the ground-truth tokens. We will walk through an example of output processing here.

In [17]:
d = data[289]
print('Raw question:', d['raw_question'])
print()
print('nP:', d['nP'])

Raw question: Mrs. Hilt measured the distance from her desk to the water fountain. It was 30 feet. How many feet will Mrs. Hilt walk on her trips to the fountain if she goes to the water fountain 4 times today?

nP: ['30' '4']


Let's assume that our model outputted `[2, 15, 16]` as the prediction.

In [18]:
pred = [2, 14, 15]
print('Model output:', pred)
print()
out_tokens = [out_vocab.idx2token[x] for x in pred]
print('Output tokens:', out_tokens)
print()
out_tokens = sub_nP(out_tokens, d['nP'])
print('Substituting nP into output tokens:', out_tokens)
print()
print('Evaluation:', evaluate_prefix_expression(out_tokens))

Model output: [2, 14, 15]

Output tokens: ['*', (0,), (1,)]

Substituting nP into output tokens: ['*', '30', '4']

Evaluation: 120.0


Note that when we evaluate our model's prediction to the ground-truth output, we compare both the predicted expression to the ground-truth expression (Validation equation accuracy) and the predicted value to the ground-truth value (Validation value accuracy).